In [1]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import scipy as sp
import sklearn
from sklearn.decomposition import TruncatedSVD
from matplotlib import pyplot as plt
from IPython.display import clear_output
import powerlaw
import tensorflow_addons as tfa
import copy
import weightwatcher as ww
import imageio
from datetime import datetime
import io
import cv2
# Suppress the powerlaw package warnings
# "powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide"
# "powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide"
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
import random

In [3]:
# Model / data parameters
num_classes = 10
inputShape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# shuffle training set and its labels accordingly
trainingIndexes = np.arange(0,y_train.shape[0]).tolist()
random.shuffle(trainingIndexes)
x_train = x_train[trainingIndexes,:,:,:]
y_train = y_train[trainingIndexes,:]

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
# LENET
model = keras.Sequential(
    [
        keras.layers.InputLayer(input_shape=inputShape),
        layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu'),
        layers.MaxPool2D(strides=2),
        layers.Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'),
        layers.MaxPool2D(strides=2),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(84, activation='relu'),
        layers.Dense(10, activation='softmax'),
    ]
)
 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 48)        38448     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 48)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1200)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               307456    
_________________________________________________________________
dense_1 (Dense)              (None, 84)                2

In [5]:
trainingSize = 2000
testSize = 10000
batch_size = 128
epochs = 15
learningRate = .001

In [6]:
#opt = keras.optimizers.SGD(learning_rate=.01, momentum = 0.9) 
opt = keras.optimizers.Adam(learning_rate=learningRate)
 
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
 
model.fit(x_train[0:trainingSize], y_train[0:trainingSize], batch_size=batch_size, epochs=epochs, validation_split=0)

Epoch 1/15
16/16 [==============================] - 3s 99ms/step - loss: 1.6177 - accuracy: 0.4930 0s - loss: 1.6177 - accuracy: 0.49
Epoch 2/15
16/16 [==============================] - 2s 95ms/step - loss: 0.5322 - accuracy: 0.8260
Epoch 3/15
16/16 [==============================] - 2s 95ms/step - loss: 0.3884 - accuracy: 0.8790
Epoch 4/15
16/16 [==============================] - 2s 93ms/step - loss: 0.2689 - accuracy: 0.9125
Epoch 5/15
16/16 [==============================] - 2s 94ms/step - loss: 0.2005 - accuracy: 0.9335
Epoch 6/15
16/16 [==============================] - 2s 95ms/step - loss: 0.1692 - accuracy: 0.9415
Epoch 7/15
16/16 [==============================] - 2s 96ms/step - loss: 0.1311 - accuracy: 0.9570
Epoch 8/15
16/16 [==============================] - 2s 95ms/step - loss: 0.0807 - accuracy: 0.9755
Epoch 9/15
16/16 [==============================] - 2s 97ms/step - loss: 0.0584 - accuracy: 0.9860
Epoch 10/15
16/16 [==============================] - 2s 96ms/step - loss: 

In [7]:
# careful! Python is dangerous :) as opposed to MATLAB, it can modify the contents of an object from inside a function even if that wasn't asked for.
# so before we begin, do a deep copy (keras clone) of the model and work with that
modelToSmooth = keras.models.clone_model(model)
modelToSmooth.build(model.input_shape) 
modelToSmooth.compile(loss=model.loss, optimizer=model.optimizer, metrics=["accuracy"])
modelToSmooth.set_weights(model.get_weights())
    
watcher = ww.WeightWatcher(model=modelToSmooth)
watcher.analyze()

INFO:weightwatcher:

python      version 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
numpy       version 1.19.5
tensforflow version 2.5.0
keras       version 2.5.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savefig': False, 'rescale': True, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'min_evals': 0, 'max_evals': None, 'plot': False, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savefig': False, 'rescale': True, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'min_evals': 0, 'max_evals': None, 'plot': False, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:LAYER: 0 16  : <class 'tensorflow.python.keras.layers.convolutional.Conv2D'>
Assuming nested distributions
INFO:weightwatcher:compare dist=lognormal R=0.7

,layer_id,name,D,M,N,alpha,alpha_weighted,best_fit,entropy,has_esd,...,num_evals,num_pl_spikes,rank_loss,rf,sigma,spectral_norm,stable_rank,sv_max,xmax,xmin
0,0,None,0.206815,1.0,32.0,5.507754,-5.340826,lognormal,0.994491,1.0,...,25.0,22.0,0.0,25.0,0.961056,0.107228,18.420618,0.327457,0.107228,0.064807
1,2,None,0.089443,32.0,48.0,2.587988,-1.472945,truncated_power_law,0.920375,1.0,...,800.0,243.0,0.0,25.0,0.101869,0.269683,110.264512,0.519310,0.269683,0.040187
2,5,None,0.073732,256.0,1200.0,3.933288,3.909370,truncated_power_law,0.968464,1.0,...,256.0,91.0,0.0,1.0,0.307492,9.860954,48.199289,3.140216,9.860954,1.999675
3,6,None,0.179069,84.0,256.0,7.092448,4.361193,lognormal,0.962117,1.0,...,84.0,14.0,0.0,1.0,1.628275,4.120088,32.424609,2.029800,4.120088,2.678033
4,7,None,0.250000,10.0,84.0,11.518981,5.557383,truncated_power_law,0.979186,1.0,...,10.0,4.0,0.0,1.0,5.259490,3.037067,6.549793,1.742718,3.037067,2.376399


In [8]:
modelToSmooth.evaluate(x_train[0:2000], y_train[0:2000])

63/63 [==============================] - 1s 10ms/step - loss: 0.0069 - accuracy: 0.9995


[0.006946598179638386, 0.9994999766349792]

In [9]:
modelToSmooth.evaluate(x_test[0:testSize], y_test[0:testSize], verbose=0)

[0.1456480324268341, 0.9628999829292297]

In [10]:
(modelito, nComponents) = watcher.unifiedSVDSmoothing(methodSelectComponents = "percentage", percent = 0.1)
print(nComponents)

INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savefig': False, 'rescale': True, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'min_evals': 0, 'max_evals': None, 'plot': False, 'mp_fit': False, 'layers': [], 'percent': 0.1}
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savefig': False, 'rescale': True, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'min_evals': 0, 'max_evals': None, 'plot': False, 'mp_fit': False, 'layers': [], 'percent': 0.1}
INFO:weightwatcher:LAYER: 0 16  : <class 'tensorflow.python.keras.layers.convolutional.Conv2D'>
INFO:weightwatcher:LAYER: 2 16  : <class 'tensorflow.python.keras.layers.convolutional.Conv2D'>
INFO:weightwatcher:LAYER: 5 4  : <class 'tensorflow.python.keras.layers.core.Dense'>
INFO:weightwatcher:LAYER: 6 4  : <class 'tensorflow.python.keras.

Unified matrix is of size (608, 608)
61


In [11]:
modelito.evaluate(x_train[0:2000], y_train[0:2000])

63/63 [==============================] - 1s 16ms/step - loss: 0.0069 - accuracy: 0.9995


[0.006946598179638386, 0.9994999766349792]